In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/TransferLearning
!ls

/content/gdrive/My Drive/TransferLearning
InceptionV3		  ResNet50	VGG16logs1		 X_train.pickle
MobileNet_v2		  test.h5	VGG16model94accuracy.h5  Y_test.pickle
MobileNetV2accuracy36.h5  testimg2.jpg	VGG19logs		 Y_train.pickle
NASNetMobile		  VGG16logs	X_test.pickle


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
from keras.models import Sequential
from keras.layers import Activation, GlobalAveragePooling2D,Dropout
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import resnet
from keras.layers import Input
import numpy as np
from keras import Model

Using TensorFlow backend.


In [0]:
tensorboard = TensorBoard(log_dir='./ResNet50', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [6]:
image_size = 224
IMG_SHAPE = (image_size, image_size, 3)
base_model = resnet.ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [7]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
CLASSES = 4
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
for layer in base_model.layers:
    layer.trainable = True

In [0]:
import pickle

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle","rb")
Y_train = pickle.load(pickle_in)

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'],
              )

In [12]:
#Fit the model
history = model.fit(X_train, Y_train,
batch_size=32,
epochs=7,
validation_data=(X_test,Y_test),
callbacks=[tensorboard])


Train on 1006 samples, validate on 607 samples


Epoch 1/7
1006/1006 [==============================] - 20s 20ms/step - loss: 0.8071 - accuracy: 0.7515 - val_loss: 704.5312 - val_accuracy: 0.2586

Epoch 2/7
1006/1006 [==============================] - 8s 8ms/step - loss: 0.3791 - accuracy: 0.8787 - val_loss: 173.7873 - val_accuracy: 0.3328
Epoch 3/7
1006/1006 [==============================] - 8s 8ms/step - loss: 0.3239 - accuracy: 0.8986 - val_loss: 4.4791 - val_accuracy: 0.5321
Epoch 4/7
1006/1006 [==============================] - 8s 8ms/step - loss: 0.0993 - accuracy: 0.9662 - val_loss: 2.1307 - val_accuracy: 0.8138
Epoch 5/7
1006/1006 [==============================] - 8s 8ms/step - loss: 0.0367 - accuracy: 0.9891 - val_loss: 6.3492 - val_accuracy: 0.5881
Epoch 6/7
1006/1006 [==============================] - 8s 8ms/step - loss: 0.0167 - accuracy: 0.9970 - val_loss: 0.1256 - val_accuracy: 0.9638
Epoch 7/7
1006/1006 [==============================] - 8s 8ms/step - loss: 0.0088 - a

In [13]:
#Score 
scores = model.evaluate(X_test, Y_test, verbose=1) 
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

607/607 [==============================] - 1s 2ms/step
Test loss: 0.41163962441182017
Test accuracy: 0.8780889511108398


In [14]:
 #precision    recall  f1-score   support
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_bool))

607/607 [==============================] - 3s 5ms/step
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       150
           1       0.90      0.87      0.89       150
           2       0.83      0.94      0.88       157
           3       0.99      0.75      0.86       150

    accuracy                           0.88       607
   macro avg       0.89      0.88      0.88       607
weighted avg       0.89      0.88      0.88       607



In [0]:
model.save("ResNet50accuracy87.h5")